In [ ]:
from dask.distributed import Client, WorkerPlugin

In [ ]:
# Close the previous scheduler and all associated workers if present
try:
    client.shutdown()
    client.close()
except:
    print('Could not shut down old client, was there any to begin with?')

In [ ]:
# First start a scheduler and worker from cli:
# $ dask-scheduler
# $ dask-worker tcp://xxx.xx.xxx.xxx:pppp --nthreads 1 --memory-limit 0 --no-nanny

# Start a client and connect it to the existing scheduler
client = Client('tcp://145.94.225.114:8786')
print('Dashboard available at', client.dashboard_link)

In [ ]:
# Worker plugin that accelerates the incoming part of the
# task graph using dask-accelerated
class WorkerOptimizer(WorkerPlugin):

    def setup(self, worker):
        print('WorkerOptimizer plugin registered')

    def transition(self, key, start, finish, **kwargs):
        print(key, '\t', start, '\t', finish)
        print(kwargs, '\n')

    def release_key(self, key, state, cause, reason, report):
        print(key, '\t', state, '\t', cause, '\t', reason, '\t', report)

    def release_dep(self, dep, state, report):
        print(dep, '\t', state, '\t', report)

In [ ]:
# Register the plugin with all current and future workers
client.register_worker_plugin(WorkerOptimizer)

In [ ]:
# Define a simple function and
# submit a future on the client

# Increment integer values by 1
def inc(x):
    return x + 1

# Y holds the future to the result
y = client.submit(inc, 10)

In [ ]:
# Print the result of the future when it is ready
y.result()

In [ ]:
# Restart the client in case something went wrong
# By default this doesnt run so the entire notebook
# can be executed without restarting the client
if False:
    client.restart()